In [1]:
import os
import numpy as np
import re
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras import models, layers

2023-09-15 08:26:45.320230: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 08:26:45.480621: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 08:26:45.481050: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 08:26:46.697660: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
num_threads = 5
os.environ["OMP_NUM_THREADS"] = "5"
os.environ["TF_NUM_INTRAOP_THREADS"] = "5"
os.environ["TF_NUM_INTEROP_THREADS"] = "5"
tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
tf.config.set_soft_device_placement(True)

In [4]:
MAX_WORDS_IN_SENTENCE = 700
WORD2VEC_DIMENSIONS = 100

In [5]:
embeddings_index = {}
with open('/home/tina/Downloads/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [6]:
def text_to_word_vectors(text, vector_space_dimension):    
    sentence_vectors = []
    for word in text.lower().split():
        if word in embeddings_index and len(sentence_vectors) < MAX_WORDS_IN_SENTENCE:
            sentence_vectors.append(embeddings_index[word])    
    if len(sentence_vectors) > 0:
        pad_len = vector_space_dimension-len(sentence_vectors)      
        if pad_len > 0:
            padding = [np.zeros_like(sentence_vectors[0])] * pad_len
            sentence_vectors = sentence_vectors + padding
    return sentence_vectors

def clean_dataframe(df):
    cleaned_df = df.copy()
    pattern = r'[^a-zA-Z\s]'
    cleaned_df['text'] = cleaned_df['text'].apply(lambda x: re.sub(pattern, '', x))    
    return cleaned_df

In [7]:
dataset = pd.read_csv('./data/reviews-dataset.csv')
dataset = clean_dataframe(dataset)
texts = dataset['text'].astype(str)
dataset['vlen'] = [len(text_to_word_vectors(t, MAX_WORDS_IN_SENTENCE)) for t in texts]
dataset = dataset[dataset.vlen > 0]
# dataset = dataset[dataset.vlen <= MAX_WORDS_IN_SENTENCE]

texts = dataset['text'].astype(str)
labels = dataset['label']

In [8]:
data = [text_to_word_vectors(text, MAX_WORDS_IN_SENTENCE) for text in texts]
data = np.array(data)
targets = np.array(labels).astype("float32")

In [9]:
test_x = data[:1500]
test_x = np.array(test_x).astype('float32')
test_x =  test_x.reshape(len(test_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
test_y = targets[:1500]

train_x = data[1500:]
train_x = np.array(train_x).astype('float32')
train_x = train_x.reshape(len(train_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
train_y = targets[1500:]

In [10]:
model = models.Sequential()
model.add(layers.Dense(50, activation="relu", input_shape=(MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS, )))  
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid")) 
model.summary()

2023-09-15 08:27:19.544456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-15 08:27:19.545620: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                3500050   
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3505201 (13.37 MB)
Trainable params: 35052

In [11]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
results = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_data=(test_x, test_y))

Epoch 1/5


2023-09-15 08:27:23.063172: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1258600000 exceeds 10% of free system memory.


141/141 [==============================] - ETA: 0s - loss: 0.7325 - accuracy: 0.5177

2023-09-15 08:27:29.805820: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 420000000 exceeds 10% of free system memory.


141/141 [==============================] - 7s 43ms/step - loss: 0.7325 - accuracy: 0.5177 - val_loss: 0.6791 - val_accuracy: 0.5640
Epoch 2/5
141/141 [==============================] - 6s 40ms/step - loss: 0.6768 - accuracy: 0.5809 - val_loss: 0.6227 - val_accuracy: 0.6707
Epoch 3/5
141/141 [==============================] - 6s 42ms/step - loss: 0.6304 - accuracy: 0.6225 - val_loss: 0.5656 - val_accuracy: 0.7033
Epoch 4/5
141/141 [==============================] - 6s 42ms/step - loss: 0.5754 - accuracy: 0.6694 - val_loss: 0.5574 - val_accuracy: 0.7087
Epoch 5/5
141/141 [==============================] - 6s 43ms/step - loss: 0.5129 - accuracy: 0.7150 - val_loss: 0.5707 - val_accuracy: 0.7207


In [12]:
model.save('./data/keras-w2v-recipe-50d.mdl')
loaded_model = models.load_model('./data/keras-w2v-recipe-50d.mdl')
scores = loaded_model.evaluate(test_x, test_y, verbose=0)
print("Loaded Model Accuracy: %.2f%%" % (scores[1] * 100))

INFO:tensorflow:Assets written to: ./data/keras-w2v-recipe-50d.mdl/assets


INFO:tensorflow:Assets written to: ./data/keras-w2v-recipe-50d.mdl/assets
2023-09-15 08:27:58.422945: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 420000000 exceeds 10% of free system memory.


Loaded Model Accuracy: 72.07%


In [13]:
scores = loaded_model.evaluate(test_x, test_y, verbose=0)
print("Loaded Model Accuracy: %.2f%%" % (scores[1] * 100))

2023-09-15 08:28:00.637547: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 420000000 exceeds 10% of free system memory.


Loaded Model Accuracy: 72.07%
